Test the saved model using pickle

In [1]:
import pickle
load_cnn_mech = pickle.load(open('small_neural_net_test_etensor_feb25_2018_n80.sav'))
load_cnn_keff = pickle.load(open('small_neural_net_test_keff_feb25_2018_n80.sav'))


/Users/Santiago/anaconda2/lib/python2.7/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator MLPRegressor from version 0.18.1 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [2]:
import numpy as np
import pandas as pd

def data_extract_txt(table_flat_list, etensor_flat_list, keff_flat_list, file_iter):
    #Open the data file
    f_test = open('mesoscale_ga_data_feb_17_datatest_n10_iter_%i.txt' % file_iter)

    #append all lines to a list, index them and 
    #find the lines that contain the 'Next Table' for a particle table
    # find the lines with 'Etensor_mat' and K_eff_mat to deliminate those matrices

    #lists and counters
    lines_list = []
    counter_next_table = 0
    counter_etensor = 0
    counter_keff = 0
    next_table_list = []
    etensor_list = []
    keff_list = []

    #find the relevant line numbers
    for line in f_test.readlines():
        lines_list.append(line)
        if 'Next Table' in line:
            next_table_list.append(counter_next_table)
        if 'Etensor_mat' in line:
            etensor_list.append(counter_etensor)
        if 'K_eff_mat' in line:
            keff_list.append(counter_keff)

        #increase the counters
        counter_next_table += 1
        counter_etensor += 1
        counter_keff += 1




    #Extract the particle table - from the start of the the table list to the beginning of the Etensor list
    num_structures = len(next_table_list)

    fiber_table1 = np.zeros((150,4))
    etensor1 = np.zeros((6,6))
    keff1 = np.zeros((3,3))

    for ii in range (0, num_structures):
        #Determine the relevant ranges
        #start and stops for the particle table
        table_start = next_table_list[ii]
        table_stop = etensor_list[ii]
        #the etensor
        etensor_start = etensor_list[ii] + 1
        etensor_stop = keff_list[ii] - 1
        #the k tensor
        keff_start = keff_list[ii] + 1
        if (ii == (num_structures -1)):
            keff_stop = len(lines_list) - 2
        else:
            keff_stop = next_table_list[ii+1] - 1

        #Contruction of the Particle Table

        #first element in the table
        table_1_split = lines_list[table_start].split()
        #first element
        parse1 = table_1_split[2].split("[")
        parse2 = parse1[1].split(",")
        float1 = float(parse2[0])
        float2 = float(table_1_split[3].split(",")[0])
        float3 = float(table_1_split[4].split(",")[0])
        float4 = float(table_1_split[5].split("]")[0])

        fiber_table1[0] = np.array([float1, float2, float3, float4])

        #all other elements in the table
        count_table = 1
        for jj in range(table_start+1, table_stop):
            table_2_split = lines_list[jj].split()
            parse1 = table_2_split[0].split("[")
            parse2 = parse1[1].split(",")
            float1 = float(parse2[0])
            float2 = float(table_2_split[1].split(",")[0])
            float3 = float(table_2_split[2].split(",")[0])
            float4 = float(table_2_split[3].split("]")[0])

            fiber_table1[count_table] = np.array([float1, float2, float3, float4])

            count_table += 1

        #Construction of the Etensor
        count_tensor = 0
        for jj in range(etensor_start, etensor_stop):
            tensor_1_split = lines_list[jj].split()
            parse1 = tensor_1_split[0].split("[")
            parse2 = parse1[1].split(",")
            float1 = float(parse2[0])
            float2 = float(tensor_1_split[1].split(",")[0])
            float3 = float(tensor_1_split[2].split(",")[0])
            float4 = float(tensor_1_split[3].split(",")[0])
            float5 = float(tensor_1_split[4].split(",")[0])
            float6 = float(tensor_1_split[5].split("]")[0])

            etensor1[count_tensor] = np.array([float1, float2, float3, float4, float5, float6])

            count_tensor += 1

        #Construction of the Keff
        count_keff = 0
        for jj in range(keff_start, keff_stop):
            keff_1_split = lines_list[jj].split()
            parse1 = keff_1_split[0].split("[")
            parse2 = parse1[1].split(",")
            float1 = float(parse2[0])
            float2 = float(keff_1_split[1].split(",")[0])
            float3 = float(keff_1_split[2].split("]")[0])


            keff1[count_keff] = np.array([float1, float2, float3])

            count_keff += 1


        #Flatten the table, etensor, and keff to prepare for input into the data frame
        table_flat_list.append(fiber_table1.flatten())
        etensor_flat_list.append(etensor1.flatten())
        keff_flat_list.append(keff1.flatten())
    

In [3]:
#iterator for the txt file
table_flat_list = []
etensor_flat_list = []
keff_flat_list = []

for file_iter in range(1,5):

    data_extract_txt(table_flat_list, etensor_flat_list, keff_flat_list, file_iter)



In [4]:
#flatten inputs and outputs into one big data list
pd_data_list = []

num_structures = len(table_flat_list)

for ii in range(0, num_structures):
    list_line = np.array([np.array(table_flat_list[ii]), np.array(etensor_flat_list[ii]), np.array(keff_flat_list[ii])])
    
    #append to list
    pd_data_list.append(list_line)

#create pandas data frame
data_frame_list = pd.DataFrame(pd_data_list, columns = ['Particle Table', 'Etensor', 'Keff'], dtype='float')



In [5]:
from sklearn.model_selection import train_test_split

X = data_frame_list['Particle Table']
Y_mech = data_frame_list['Etensor']
Y_therm = data_frame_list['Keff']

# x_train, x_test, y_train, y_test = train_test_split(X, Y)

#convert the list of arrays back to a numpy array!!! BITHESSSSS
# np1 = np.vstack(x_train)

In [6]:
predictions_mech = load_cnn_mech.predict(np.vstack(X))
predictions_therm = load_cnn_keff.predict(np.vstack(X))

In [7]:
def error_func(predictions, test_data):
    error = 0.0
    for ii in range(0, len(test_data)):
        error += np.linalg.norm((predictions[ii]-test_data[ii]))/np.linalg.norm(test_data[ii])
        
    return error

In [8]:
error_func(predictions_mech, np.vstack(Y_mech))

35.999015116695631

In [9]:
error_func(predictions_therm, np.vstack(Y_therm))

29.741590453438565

In [15]:
predictions_mech

array([[  1.43263515e+07,   3.95238958e+06,   3.95239631e+06, ...,
          3.47478828e+03,  -1.41755283e+03,   5.23969616e+06],
       [  1.43263515e+07,   3.95238958e+06,   3.95239631e+06, ...,
          3.47478828e+03,  -1.41755283e+03,   5.23969616e+06],
       [  1.43263515e+07,   3.95238958e+06,   3.95239631e+06, ...,
          3.47478828e+03,  -1.41755283e+03,   5.23969616e+06],
       ..., 
       [  1.43263515e+07,   3.95238958e+06,   3.95239631e+06, ...,
          3.47478828e+03,  -1.41755283e+03,   5.23969616e+06],
       [  1.43263515e+07,   3.95238958e+06,   3.95239631e+06, ...,
          3.47478828e+03,  -1.41755283e+03,   5.23969616e+06],
       [  1.43263515e+07,   3.95238958e+06,   3.95239631e+06, ...,
          3.47478828e+03,  -1.41755283e+03,   5.23969616e+06]])

In [16]:
Y_mech

0     [515323206234.0, 142410309173.0, 142295255807....
1     [515906164747.0, 142625196470.0, 142221215287....
2     [520355550893.0, 143678521742.0, 142997571282....
3     [529690596238.0, 146720247572.0, 146827158820....
4     [539427682806.0, 148189965084.0, 148247309752....
5     [536371885475.0, 147493749061.0, 148463254256....
6     [510362059412.0, 140897370963.0, 140938666443....
7     [519970647542.0, 143634118520.0, 144187952850....
8     [525418591278.0, 145544027888.0, 146277202998....
9     [510362059412.0, 140897370963.0, 140938666443....
10    [515323206234.0, 142410309173.0, 142295255807....
11    [515906164747.0, 142625196470.0, 142221215287....
12    [530077419177.0, 146352385512.0, 146728886298....
13    [546206243372.0, 150572952686.0, 151091436775....
14    [539427682806.0, 148189965084.0, 148247309752....
15    [508609207344.0, 140675865290.0, 140479925744....
16    [538112886801.0, 147912185650.0, 148395906640....
17    [505649775104.0, 140518112258.0, 140120300